In [2]:
import pandas as pd
#import ydata_profiling as yp
# data preprocessing
from sklearn.preprocessing import StandardScaler
# data splitting
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from sklearn import neighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error 
from math import sqrt
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix,accuracy_score,roc_curve,classification_report,matthews_corrcoef,roc_auc_score
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [8]:
data = pd.read_csv("/home/cloud/Datasets/Hotel Reservations_data_features.csv")

## Train test split the data

In [10]:
def train_test_split_and_scale(data):
    y = data["booking_status"]
    x = data.drop("booking_status", axis=1)
    features = list(x.columns)

    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=0.20, random_state=0
    )

    scaler = MinMaxScaler()

    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)


    x_train_scaled = pd.DataFrame(x_train_scaled, columns=features, index=x_train.index)
    x_test_scaled = pd.DataFrame(x_test_scaled, columns=features, index=x_test.index)

    return x_train_scaled, x_test_scaled, y_train, y_test, features, scaler

In [12]:
x_train, x_test, y_train, y_test,features, scaler = train_test_split_and_scale(data)

In [14]:
x_train.head(10)

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,repeated_guest,avg_price_per_room,no_of_previous_bookings_not_canceled,no_of_special_requests,...,arrival_month_3,arrival_month_4,arrival_month_5,arrival_month_6,arrival_month_7,arrival_month_8,arrival_month_9,arrival_month_10,arrival_month_11,arrival_month_12
26817,0.5,0.000000,0.285714,0.176471,0.0,0.550790,0.0,0.158981,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7537,0.5,0.000000,0.000000,0.176471,0.0,0.623025,0.0,0.175926,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6217,0.5,0.000000,0.142857,0.117647,0.0,0.525959,0.0,0.166667,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
22883,0.5,0.000000,0.285714,0.117647,0.0,0.352144,0.0,0.172370,0.0,0.2,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
28669,0.5,0.000000,0.000000,0.176471,0.0,0.171558,0.0,0.128389,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31629,0.5,0.000000,0.000000,0.117647,0.0,0.715576,0.0,0.222222,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
957,0.5,0.222222,0.285714,0.058824,0.0,0.128668,0.0,0.423333,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
937,0.0,0.222222,0.000000,0.176471,0.0,0.090293,0.0,0.012352,0.0,0.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3911,0.5,0.000000,0.142857,0.176471,0.0,0.040632,0.0,0.260926,0.0,0.2,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
15346,0.5,0.000000,0.000000,0.117647,0.0,0.011287,0.0,0.160370,0.0,0.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## Fit and evaluate the model

In [30]:
def fit_and_evaluate_knn(x_train, x_test, y_train, y_test, n_neighbors=5):
    knn = KNeighborsClassifier(
        n_neighbors=n_neighbors,
        metric='minkowski'  # default = euclidean
    )

    model = knn.fit(x_train, y_train)

    knn_predict = knn.predict(x_test)
    knn_prob = knn.predict_proba(x_test)[:, 1]

    knn_conf_matrix = confusion_matrix(y_test, knn_predict)
    knn_acc_score = accuracy_score(y_test, knn_predict)
    knn_mcc_score = matthews_corrcoef(y_test, knn_predict)
    knn_auc_score = roc_auc_score(y_test, knn_prob)

    print("Confusion Matrix:")
    print(knn_conf_matrix)
    print("\nAccuracy:", knn_acc_score * 100,'\n')
    print("MCC Score:", knn_mcc_score * 100,'\n')
    print("AUC Score:", knn_auc_score * 100,'\n')
    print(classification_report(y_test, knn_predict))

    return model

In [32]:
knn_model = fit_and_evaluate_knn(x_train,x_test,y_train,y_test,n_neighbors=7)

Confusion Matrix:
[[4437  417]
 [ 670 1731]]

Accuracy: 85.0172294968987 

MCC Score: 65.45573675095217 

AUC Score: 90.9899382673783 

              precision    recall  f1-score   support

           0       0.87      0.91      0.89      4854
           1       0.81      0.72      0.76      2401

    accuracy                           0.85      7255
   macro avg       0.84      0.82      0.83      7255
weighted avg       0.85      0.85      0.85      7255



In [38]:
import joblib
joblib.dump(
    {
        "model": knn_model,
        "scaler": scaler,
        "features": features
    },
    "/home/cloud/ML_Models/knn.pkl"
)

['/home/cloud/ML_Models/knn.pkl']